In [1]:
import numpy as np
import pandas as pd

In [2]:
import time
import os.path
import shutil

def backup_file(filename):
    timestr = time.strftime('%Y%m%d-%H%M%S')
    if os.path.isfile(filename):
        shutil.copy2(filename,filename+'.bak_' + timestr)
    return filename

In [3]:
#path = 'M:/My Documents/Business/Bugisiw Ltd/TaxReturn/TaxReturn_2020-2021/LLP accounts/'
taxreturn_yr='TaxReturn_2021-2022'
path = 'E:\\dtuklaptop\\e\\Users\\Mat\\python\\data\\property\\'+taxreturn_yr+'\\LLP accounts\\'
startdatestr='2021-04-06'
enddatestr='2022-04-05'

In [4]:
startdate = pd.to_datetime(startdatestr).date()
enddate = pd.to_datetime(enddatestr).date()

In [5]:
input_file_6045 = path + '6045_categorisedAndBealsReconciled.xlsx'
df6=pd.read_excel(input_file_6045,sheet_name='6045')
df6=df6[~df6.Account.isnull()]
input_file_3072 = path + '3072_categorisedAndBealsReconciled.xlsx'
df3=pd.read_excel(input_file_3072,sheet_name='3072')
df3=df3[~df3.Account.isnull()]
#df3.loc[(df3.Description.isnull()),'Description']=''

In [6]:
input_file_beals = path + 'beals_bankReconciled.xlsx'
dfB=pd.read_excel(input_file_beals,sheet_name='beals')
dfB['ReconciledBankAccount']=dfB['ReconciledBankAccount'].astype('str')

### Transform Bank data

In [7]:
df6.rename(columns={'Account':'BankAccount'},inplace=True)
df3.rename(columns={'Account':'BankAccount'},inplace=True)
df6['PostNarrative'] = df6.Property.replace(np.nan,'') + ':6045:' + df6['Memo'].replace(np.nan,'') + '/' + df6['Description'].replace(np.nan,'')
df3['PostNarrative'] = df3.Property.replace(np.nan,'') + ':3072:' + df3['Memo'].replace(np.nan,'') + '/' + df3['Description'].replace(np.nan,'')
df6['Amount']=-df6['Amount']
df3['Amount']=-df3['Amount']

### Assign Accounting codes

In [8]:
config_file = path + 'accounts-config.xlsx'
dfCfg=pd.read_excel(config_file,sheet_name='config',converters={'MT6045':str,'IV3072':str})

In [9]:
dfCfg

,Id,Category,MT6045,IV3072,Natwest,AccountDescription
0,1,CouncilTax,3110,3110,NaN,Rates
1,2,Insurance,3170,3170,NaN,Premises Insurance
2,3,AgentFees,1570,1570,NaN,Management Fees
3,4,Maintenance,3140,3140,NaN,"Repairs, renewals, maintenance"
4,5,Mortgage,3631,3631,3631.0,Other interest payable
5,6,RentalIncome,0001,0002,NaN,Rental Income
6,7,ServiceCharge,3120,3120,NaN,Property management and service charges
7,8,Accounting,9999,9999,NaN,Accounting
8,10,Drawings,7501,7501,NaN,Drawings
9,11,Deposit,9999,9999,NaN,Deposit


In [10]:
cols=['BankAccount','Date','Property','PostNarrative','Amount','Account','Account Description','Category']
journals=path + 'journals/'

for idx, row in dfCfg.iterrows():
    df6.loc[(df6.Category==row.Category),'Account']=row.MT6045
    df6.loc[(df6.Category==row.Category),'Account Description']=row.AccountDescription
    df3.loc[(df3.Category==row.Category),'Account']=row.IV3072
    df3.loc[(df3.Category==row.Category),'Account Description']=row.AccountDescription
    
    if(df6[cols][(df6.Category==row.Category)].shape[0]>0):
        writer = pd.ExcelWriter(backup_file(journals + '/' + str(row.Id) + '.' + row.Category + '_6045.xlsx'), engine='xlsxwriter',datetime_format='dd/mm/yyyy')
        df6[cols][(df6.Category==row.Category)].to_excel(writer,index=False)
        writer.save()
    if(df3[cols][(df3.Category==row.Category)].shape[0]>0):
        writer = pd.ExcelWriter(backup_file(journals + '/' + str(row.Id) + '.' + row.Category + '_3072.xlsx'), engine='xlsxwriter',datetime_format='dd/mm/yyyy')
        df3[cols][(df3.Category==row.Category)].to_excel(writer,index=False)
        writer.save()

### Generate Kingston Rd mortgage journal

In [13]:
input_file_beals = path + 'inc_exp_2022_240622_llp_all_19.xls'
dfR=pd.read_excel(input_file_beals,sheet_name='Sheet 1')
dfR=dfR[(dfR.date >= startdate)&(dfR.date <= enddate)]

C:\Users\MSTDTP21\.conda\envs\mtpython36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
dfK=dfR[(dfR.property_id==17)&(dfR.cat=='Mortgage')].copy()
dfK['BankAccount']='Natwest'
dfK['Property']='SHOP196KIN'
dfK['PostNarrative']='SHOP196KIN:Natwest:196 Kingston Rd Mortgage'
dfK['Account']=dfCfg['Natwest'][dfCfg.Category=='Mortgage'].iloc[0]
dfK['Account Description']=dfCfg['AccountDescription'][dfCfg.Category=='Mortgage'].iloc[0]
dfK['Category']='Mortgage'
dfK.rename(columns={'date':'Date'},inplace=True)
dfK.rename(columns={'amount':'Amount'},inplace=True)

writer = pd.ExcelWriter(backup_file(journals + '/' + str(dfCfg['Id'][dfCfg.Category=='Mortgage'].iloc[0]) + '.Mortgage_Natwest.xlsx'), engine='xlsxwriter',datetime_format='dd/mm/yyyy')
dfK[cols].to_excel(writer,index=False)
writer.save()


### Assign Owners - not currently used

In [ ]:
propertyownermap = {
'RSA':'Mathew Tucker/Ivana Valentino',    
'321LON':'Mathew Tucker',
'F1321LON':'Mathew Tucker',
'F2321LON':'Mathew Tucker',
'F3321LON':'Mathew Tucker',
'F4321LON':'Mathew Tucker',
'169FAW':'Mathew Tucker/Ivana Valentino',
'F1169FAW':'Mathew Tucker',
'F2169FAW':'Mathew Tucker',
'F3169FAW':'Mathew Tucker',
'171FAW':'Mathew Tucker/Ivana Valentino',
'F1171FAW':'Mathew Tucker',
'F2171FAW':'Mathew Tucker',
'F3171FAW':'Ivana Valentino',
'163FRA':'Mathew Tucker',
'F2163FRA':'Mathew Tucker',
'F3163FRA':'Mathew Tucker',
'F4163FRA':'Mathew Tucker',
'196KIN':'Mathew Tucker/Ivana Valentino',
'SHOP196KIN':'Mathew Tucker/Ivana Valentino',
'196AKIN':'Mathew Tucker',
'196BKIN':'Ivana Valentino',
'23BHAM':'Mathew Tucker',
'23CHAM':'Ivana Valentino',
'F58ALH':'Mathew Tucker',
'F68ALH':'Mathew Tucker',
'F78ALH':'Ivana Valentino',
'F88ALH':'Ivana Valentino',
'F1746ALH':'Mathew Tucker',
'F1846ALH':'Mathew Tucker',
'F1946ALH':'Mathew Tucker',
'F2046ALH':'Mathew Tucker',
'F2146ALH':'Ivana Valentino',
'F2246ALH':'Ivana Valentino',
'F2346ALH':'Ivana Valentino',
'F31214ALH':'Ivana Valentino',
'F41214ALH':'Mathew Tucker',
'F51214ALH':'Ivana Valentino',
'F71214ALH':'Mathew Tucker',
'F111214ALH':'Mathew Tucker',
'F101214ALH':'Ivana Valentino',
'F141214ALH':'Mathew Tucker',
'F161214ALH':'Ivana Valentino',
'F11618ALH':'Mathew Tucker',
'F31618ALH':'Ivana Valentino',
'F61618ALH':'Mathew Tucker',
'F71618ALH':'Ivana Valentino',
'F111618ALH':'Mathew Tucker',
'F121618ALH':'Ivana Valentino',
'F131618ALH':'Mathew Tucker',
'F161618ALH':'Ivana Valentino'
}

In [ ]:
def assign_owners(df_bank):
    for index, row in df_bank[(df_bank.Property.notnull())].iterrows():
        df_bank.at[index,'Owner']=propertyownermap[row.Property]

In [ ]:
assign_owners(df6)
assign_owners(df3)